# Chapter 11: PID Controller Design Tuning and Troubleshooting

## Example 11.1: Direct_synthesis_for_PID.sce

In [ ]:
clear
clc
//Example 11.1
disp('Example 11.1')
//(a) Desired closed loop gain=1 and tau=[1 3 10]
s=%s;
tauc=[1 3 10]';
tau1=10;tau2=5;K=2;theta=1; //Time delay
Y_Ysp=(1)./(tauc*s+1); //Y/Ysp=delay/(tau*s+1)  Eqn 11-6
//delay=(1-theta/2*s+theta^2/10*s^2-theta^3/120*s^3)/(1+theta/2*s+theta^2/10*s^2+theta^3/120*s^3);//Third order pade approx
delay=(1-theta/2*s)/(1+theta*s/2);//first order Pade approx
G=(K)./((tau1*s+1)*(tau2*s+1))*delay;
G_tilda=G//Model transfer function
//Eqn-11-14
Kc=1/K*(tau1+tau2)./(tauc+theta);tauI=tau1+tau2;tauD=tau1*tau2/(tau1+tau2);
Gc=Kc*(1+1/tauI/s+tauD*s); //PID without derivative filtering
G_CL=syslin('c',Gc/delay*G./(1+Gc*G));//closed loop transfer function
t=0:160;
y=csim('step',t,G_CL);
//plot(t,y)
t_d=81:160;
G_CL_dist=syslin('c',G/delay./(1+Gc*G));//closed loop wrt disturbance
u_d=[0 ones(1,length(t_d)-1)]
y_d=csim('step',t_d,G_CL_dist);
y(:,81:160)=y(:,81:160)+y_d
plot(t,y)
xgrid()
xtitle('Ex-11.1 Correct Model','Time(min)','y(t)');
a=legend('$	au_c=1$','$	au_c=3$','$	au_c=10$',position=4);
a.font_size=2;
a=get('current_axes');b=a.title;b.font_size=5;c=a.x_label;c.font_size=5;
c=a.y_label;c.font_size=5;
mprintf('
                  tauc=1      tauc=3      tauc=10')
mprintf('
 Kc(K_tilda=2) %10f    %f      %f',Kc');
//Simulation for model with incorrect gain
scf()
K_tilda=0.9
//Eqn-11-14
Kc=1/K_tilda*(tau1+tau2)./(tauc+theta);tauI=tau1+tau2;tauD=tau1*tau2/(tau1+tau2);
Gc=Kc*(1+1/tauI/s+tauD*s) 
mprintf('
 Kc(K_tilda=0.9) %10f    %f      %f',Kc');
mprintf('
 tauI %20f    %f      %f',tauI*ones(1,3));
mprintf('
 tauD %20f    %f      %f',tauD*ones(1,3));
G_CL=syslin('c',Gc*G./(1+Gc*G));//closed loop transfer function
t=0:160;
y=csim('step',t,G_CL);
t_d=81:160;
G_CL_dist=syslin('c',G./(1+Gc*G));//closed loop wrt disturbance
y_d=csim('step',t_d,G_CL_dist);
y(:,81:160)=y(:,81:160)+y_d
plot(t,y)
xgrid()
xtitle('Ex-11.1 Model with incorrect gain','Time(min)','y(t)');
a=legend('$	au_c=1$','$	au_c=3$','$	au_c=10$',position=4);
a.font_size=2;
a=get('current_axes');b=a.title;b.font_size=5;c=a.x_label;c.font_size=5;
c=a.y_label;c.font_size=5;
mprintf('
 
There is a slight mis-match between graphs from scilab code
...
and those given in the book because of Pade approx which is very bad
...
for delay being 1. It works only for small delays. Scilab does 
...
not handle continuous delays and hence this problem cannot 
...
be circumvented' )

## Example 11.3: PI_and_PID_control_of_liquid_storage_tank.sce

In [ ]:
clear
clc
//Example 11.3
disp('Example 11.3')
//(a)
K=0.2;theta=7.4;tauc=[8 15]';
Kc1=1/K*(2*tauc+theta)./(tauc+theta).^2; //Row M
Kc2=1/K*(2*tauc+theta)./(tauc+theta/2).^2; //Row N
tauI=2*tauc+theta;
tauD=(tauc*theta+theta^2/4)./(2*tauc+theta);
mprintf('               Kc          tauI             tauD')
mprintf('
PI(tauC=8)   %f    %f     %f',Kc1(1),tauI(1),0)
mprintf('
PI(tauC=15)   %f    %f     %f',Kc1(2),tauI(2),0)
mprintf('
PID(tauC=8)   %f    %f     %f',Kc2(1),tauI(1),tauD(1))
mprintf('
PID(tauC=15)   %f    %f     %f',Kc2(2),tauI(2),tauD(2))
s=%s;
//delay=(1-theta/2*s+theta^2/10*s^2-theta^3/120*s^3)/(1+theta/2*s+theta^2/10*s^2+theta^3/120*s^3);//Third order pade approx
delay=(1-theta/2*s+theta^2/10*s^2)/(1+theta/2*s+theta^2/10*s^2);//second order pade approx
//delay=(1-theta/2*s)/(1+theta/2*s);//first order pade approx
G=K*delay/s;
Gc1=Kc1.*(1+(1)./tauI/s)
Gc2=Kc2.*(1+(1)./tauI/s+tauD*s./(0.1*tauD*s+1));//PID with derivative filtering
G_CL1=syslin('c',Gc1*G./(1+Gc1*G));
G_CL2=syslin('c',Gc2*G./(1+Gc2*G));
t=0:300;
y1=csim('step',t,G_CL1);
y2=csim('step',t,G_CL2);
y1(:,1:theta)=0;//accounting for time delay--this is required otherwise
//an unrealistic inverse response is seen due to the pade approx
y2(:,1:theta)=0;
t_d=151:300;
G_CL_dist1=syslin('c',G./(1+Gc1*G));//closed loop wrt disturbance
G_CL_dist2=syslin('c',G./(1+Gc2*G));//closed loop wrt disturbance
y_d1=csim('step',t_d,G_CL_dist1);
y_d1(:,1:theta)=0;//accounting for time delay
y_d2=csim('step',t_d,G_CL_dist2);
y_d2(:,1:theta)=0;//accounting for time delay
y1(:,t_d)=y1(:,t_d)+y_d1;
y2(:,t_d)=y2(:,t_d)+y_d2;
//plot(t,y1)
//xgrid()
//xtitle('Ex-11.3 PI control','Time(min)','y(t)');
//a=legend('$	au_c=8$','$	au_c=15$',position=1);
//a.font_size=2;
//a=get('current_axes');b=a.title;b.font_size=5;c=a.x_label;c.font_size=5;
//c=a.y_label;c.font_size=5;
//scf()
//
//plot(t,y2)
//xgrid()
//xtitle('Ex-11.3 PID control','Time(min)','y(t)');
//a=legend('$	au_c=8$','$	au_c=15$',position=1);
//a.font_size=2;
//a=get('current_axes');b=a.title;b.font_size=5;c=a.x_label;c.font_size=5;
//c=a.y_label;c.font_size=5;
mprintf('

There is uncertainty as to whether PID with derivative filtering
...
to be used or not. Since one gets results by using PID with filtering
...
it has been used here. Note that pade approx for delay=7.4
...
is totally wrong because it is too gross an approx but we have no
...
other way of making delay approx so we have to live with it.

...')
//Part (b) Routh Array testing
//For frequency response refer to ch-13 for Bode Plots
G=(1-theta*s)/s;
poly_PI=Gc1*G;//denom(G_CL1);//G*Gc for PI controller
poly_PID=Gc2*G;//G*Gc for PID controller
Routh1=routh_t(poly_PI(1,1)/1,poly(0,'K')); // produces routh table for polynomial 1+Kc*poly
disp(Routh1,'Routh1=')
Kmax1=roots(numer(Routh1(1,1)));
Routh2=routh_t(poly_PI(2,1)/1,poly(0,'K')); // produces routh table for polynomial 1+Kc*poly
disp(Routh2,'Routh2=')
Kmax2=roots(numer(Routh2(1,1)));
Routh3=routh_t(poly_PID(1,1)/1,poly(0,'K')); // produces routh table for polynomial 1+Kc*poly
disp(Routh3,'Routh3=')
//Kmax3=roots(numer(Routh3(1,1)));
Routh4=routh_t(poly_PID(2,1)/1,poly(0,'K')); // produces routh table for polynomial 1+Kc*poly
disp(Routh4,'Routh4=')
//Kmax4=roots(numer(Routh4(1,1)));
mprintf('
 Kmax should be less than %f and %f 
 for tauc=8 and 15 respectively for PI system to be stable',Kmax1,Kmax2)
mprintf('

Answers to Kmax for PID controller using 
...
Routh Array in the book are wrong. This can be easily 
...
checked from Routh3 and Routh4 which are displayed
')
mprintf('

For frequency response refer to ch-13 for Bode Plots
')

## Example 11.4: IMC_for_lag_dominant_model.sce

In [ ]:
clear
clc
//Example 11.4
disp('Example 11.4')
s=%s;
theta=1;tau=100;K=100;
delay=(1-theta/2*s+theta^2/10*s^2-theta^3/120*s^3)/(1+theta/2*s+theta^2/10*s^2+theta^3/120*s^3);//Third order pade approx
G=K*delay/(tau*s+1);
//(a)
tauca=1;
Kc1=1/K*tau/(tauca+theta);taui1=tau;
//(b)
taucb=2;Kstar=K/tau;
Kc2=1/Kstar*(2*taucb+theta)./(taucb+theta).^2; //Row M
taui2=2*taucb+theta;
//(c)
taucc=1;
Kc3=Kc1;taui3=min(taui1,4*(taucc+theta))
//(d)
//Kc4=0.551;taui4=4.91;
//Chen and Seborg settings given in Second Edition of book
mprintf('                   Kc          tauI')
mprintf('
IMC   %20f    %f',Kc1,taui1)
mprintf('
Integrator approx   %-5f    %f',Kc2,taui2)
mprintf('
Skogestad   %15f    %f',Kc3,taui3)
//mprintf('
DS-d   %20f    %f',Kc4,taui4)  
Gc=[Kc1 Kc2 Kc3]'.*(1+(1)./([taui1 taui2 taui3]'*s))
G_CL=syslin('c',Gc*G./(1+Gc*G));
t=0:0.1:20;
y=csim('step',t,G_CL);
y(:,1:theta/0.1)=0;//accounting for time delay--this is required otherwise
//an unrealistic inverse response is seen due to the pade/taylor approx
plot(t,y);
xgrid()
xtitle('Ex-11.4 Tracking problem','Time(min)','y(t)');
a=legend('IMC','Integrator approx','Skogestad',position=4);
a.font_size=2;
a=get('current_axes');b=a.title;b.font_size=5;c=a.x_label;c.font_size=5;
c=a.y_label;c.font_size=5;
scf()
t=0:0.1:60;
G_CL_dist=syslin('c',G./(1+Gc*G));//closed loop wrt disturbance
yd=csim('step',t,G_CL_dist);
yd(:,1:theta/0.1)=0;//accounting for time delay--this is required otherwise
//an unrealistic inverse response is seen due to the pade/taylor approx
plot(t,yd);
xgrid()
xtitle('Ex-11.4 Disturbance rejection','Time(min)','y(t)');
a=legend('IMC','Integrator approx','Skogestad',position=4);
a.font_size=2;
a=get('current_axes');b=a.title;b.font_size=5;c=a.x_label;c.font_size=5;
c=a.y_label;c.font_size=5;

## Example 11.5: PI_controller_IMC_ITAE.sce

In [ ]:
clear
clc
//Example 11.5
disp('Example 11.5')
K=1.54;theta=1.07;tau=5.93;
//(a)
tauca=tau/3;
Kc1=1/K*tau/(tauca+theta);taui1=tau; //Table 11.1
//(b)
taucb=theta;
Kc2=1/K*tau/(taucb+theta);taui2=tau; //Table 11.1
//(c)
//Table 11.3
Y=0.859*(theta/tau)^(-0.977);Kc3=Y/K;
taui3=tau*inv(0.674*(theta/tau)^-0.680); 
//(d)
//Table 11.3
Kc4=1/K*0.586*(theta/tau)^-0.916;taui4=tau*inv(1.03-0.165*(theta/tau)); 
mprintf('                   Kc          tauI')
mprintf('
IMC(tauC=tau/3)   %f    %f',Kc1,taui1)
mprintf('
IMC(tauC=theta)   %-5f    %f',Kc2,taui2)
mprintf('
ITAE(disturbance)  %f    %f',Kc3,taui3)
mprintf('
ITAE(set point)  %10f    %f',Kc4,taui4)

## Example 11.6: Controller_with_two_degrees_of_freedom.sce

In [ ]:
clear
clc
//Example 11.6
disp('Example 11.6')
//Drawing on example 11.4
s=%s;
theta=1;tau=100;K=100;
delay=(1-theta/2*s+theta^2/10*s^2-theta^3/120*s^3)/(1+theta/2*s+theta^2/10*s^2+theta^3/120*s^3);//Third order pade approx
G=K*delay/(tau*s+1);
Kc=0.556;taui=5;
Gc=Kc.*(1+(1)./([taui]*s))
G_CL=syslin('c',Gc*G./(1+Gc*G));
t=0:0.1:20;
y1=csim('step',t,G_CL);
y1(:,1:theta/0.1)=0;//accounting for time delay--this is required otherwise
//an unrealistic inverse response is seen due to the pade/taylor approx
beta=0.5;
G_CL2=syslin('c',(Gc+beta-1)*G./(1+Gc*G));//This can be obtained on taking
//laplace transform of eqn 11-39 and making a block diagram 
//In Eqn 11-39 p refers to input to the process
t=0:0.1:20;
y2=csim('step',t,G_CL2);
y2(:,1:theta/0.1)=0;//accounting for time delay--this is required otherwise
//an unrealistic inverse response is seen due to the pade/taylor approx
plot(t,[y1; y2]);
xgrid()
xtitle('Ex-11.3 Tracking problem','Time(min)','y(t)');
a=legend('$eta=1$','$eta=0.5$',position=4);
a.font_size=2;
a=get('current_axes');b=a.title;b.font_size=5;c=a.x_label;c.font_size=5;
c=a.y_label;c.font_size=5;
//Note that there is a slight mis-match between the plots obtained from scilab code
//and that of the book because of third order pade approximation
//The plots in the book have been produced using advanced proprietary software
//which supports using exact delays while scilab does not have that functionality

## Example 11.7: Continuous_cycling_method.sce

In [ ]:
clear
clc
//Example 11.7
disp('Example 11.7')
K=2;theta=1;tau1=10;tau2=5; //Model parameters
s=%s;
delay=(1-theta/2*s+theta^2/10*s^2-theta^3/120*s^3)/(1+theta/2*s+theta^2/10*s^2+theta^3/120*s^3);//Third order pade approx
G=K*delay/((tau1*s+1)*(tau2*s+1));
Ku=[8.01]';//Trials for various values of Ku can be done by changing Ku
G_CL_trial=syslin('c',G*Ku./(1+G*Ku))
t=0:0.1:100;
y=csim('step',t,G_CL_trial);
plot(t,y)
xtitle('Ex-11.7 Finding ultimate gain Ku','Time(min)','y(t)');
a=legend('Closed loop test',position=4);
a.font_size=2;
a=get('current_axes');b=a.title;b.font_size=5;c=a.x_label;c.font_size=5;
c=a.y_label;c.font_size=5;
//There isnot a sustained oscillation for Ku=7.88, in our simulation because
//we are using a third order Pade Approx for delay.  But still we go ahead with it
//so that it matches with the example values. Our simulations give Ku=8
Ku=7.88;Pu=11.66;
//(a) Table 11.4 ZN
Kc1=0.6*Ku;taui1=Pu/2;tauD1=Pu/8;
//(b) Table 11.4 TL
Kc2=0.45*Ku;taui2=Pu*2.2;tauD2=Pu/6.3;
//(c) DS method
tauc=3;
Kc3=1/K*(tau1+tau2)/(tauc+theta);taui3=tau1+tau2;tauD3=tau1*tau2/(tau1+tau2);
mprintf('        Kc          tauI        tauD')
mprintf('
ZN   %f    %f      %f',Kc1,taui1,tauD1)
mprintf('
TL   %f    %f      %f',Kc2,taui2,tauD2)
mprintf('
DS   %f    %f      %f',Kc3,taui3,tauD3)
// Now we compare the performance of each controller setting
Gc1=Kc1.*(1+(1)./taui1/s+tauD1*s)
Gc2=Kc2.*(1+(1)./taui2/s+tauD2*s)
Gc3=Kc3.*(1+(1)./taui3/s+tauD3*s)
Gc=[Gc1 Gc2 Gc3]';
G_CL=syslin('c',Gc*G./(1+Gc*G));
t=0:160;
y=csim('step',t,G_CL);
y(:,1:theta)=0;//accounting for time delay--this is required otherwise
//an unrealistic inverse response is seen due to the pade/taylor approx
t_d=81:160;
G_CL_dist=syslin('c',G./(1+Gc*G));//closed loop wrt disturbance
yd=csim('step',t_d,G_CL_dist);
yd(:,1:theta)=0;//accounting for time delay
y(:,t_d)=y(:,t_d)+yd;
scf();
plot(t,y)
xgrid()
xtitle('Ex-11.7 Comparison of 3 controllers','Time(min)','y(t)');
a=legend('Ziegler-Nichols','Tyerus-Luyben','Direct Synthesis',position=4);
a.font_size=2;
a=get('current_axes');b=a.title;b.font_size=5;c=a.x_label;c.font_size=5;
c=a.y_label;c.font_size=5;
mprintf('

There is slight mismatch between scilab simulation
...
and book simulation due to Pade approx
')

## Example 11.8: Reaction_curve_method.sce

In [ ]:
clear
clc
//Example 11.8
disp('Example 11.8')
S=(55-35)/(7-1.07);//%/min
delta_p=43-30;//%
R=S/delta_p;//min^-1
delta_x=55-35;//%
K=delta_x/delta_p;
theta=1.07;//min
tau=7-theta;//min
mprintf('
The resulting process model is with delay of 1.07 min
')
s=%s;
G=K/(tau*s+1);
disp(G,'G=')